In [1]:
#export
import pickle, gzip, math, os, time, shutil, torch, matplotlib as mpl, numpy as np, matplotlib.pyplot as plt
import fastcore.all as fc
from collections.abc import Mapping
from pathlib import Path
from operator import attrgetter, itemgetter
from functools import partial

from torch import tensor, nn, optim
from torch.utils.data import DataLoader, default_collate
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from datasets import load_dataset, load_dataset_builder

try:
    from .datasets import *
except:
    from src.miniai.datasets import *

In [6]:
# Using our dataset from before
# we will be using a dataset dictionary from huggingface
x, y = 'image', 'label'
name = "fashion_mnist"
dsd = load_dataset(name)

Reusing dataset fashion_mnist (/Users/diegomedina-bernal/.cache/huggingface/datasets/fashion_mnist/fashion_mnist/1.0.0/8d6c32399aa01613d96e2cbc9b13638f359ef62bb33612b077b4c247f6ef99c1)
100%|██████████| 2/2 [00:00<00:00, 72.61it/s]


In [7]:
@inplace
def transformi(b):
    b[x] = [torch.flatten(TF.to_tensor(o)) for o in b[x]]

In [8]:
bs = 256
tds = dsd.with_transform(transformi) # transformi is an inplace function

In [12]:
dsd['train']['image'][0]

In [14]:
tds['train']['image'][0].shape

torch.Size([784])

In [19]:
#export
class DataLoaders:
    """This class is a wrapper for the dataloaders, it specifically takes in a dataset dictionary from huggingface in this case"""
    def __init__(self, *dls):
        self.train, self.valid = dls[:2]

    @classmethod
    def from_dd(cls, dd, batch_size, as_tuple=False):
        """a dd is a dataset dictionary from huggingface. calling values return the dictionary values which are the datasets in the object"""
        return cls(*[DataLoader(ds, batch_size, collate_fn=collate_dict(ds)) for ds in dd.values()])

In [23]:
dls = DataLoaders.from_dd(tds, bs)
dt = dls.train
xb, yb = next(iter(dt))
xb.shape, yb[:10]

(torch.Size([256, 784]), tensor([9, 0, 0, 3, 0, 2, 7, 2, 5, 5]))